# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [86]:
# Import cities file as DataFrame
weather_data = pd.read_csv("output_data/cities.csv")
weather_data.head()


,City,Country,Latitude,Longitude,Temperature,Humidity,Wind,Cloudiness,Datetime
0,albany,US,42.60,-73.97,39.79,33,24.16,40,1587069002
1,yar-sale,RU,66.83,70.83,16.97,98,10.83,97,1587069015
2,ushuaia,AR,-54.80,-68.30,57.20,58,4.70,99,1587069008
3,gardabani,GE,41.46,45.09,46.40,70,34.45,82,1587069121
4,esperance,AU,-33.87,121.90,51.80,87,6.93,0,1587069048


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [87]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [88]:
# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [89]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [103]:
weather_data_ideal = weather_data.loc[(weather_data['Temperature'] >= 62) & (weather_data['Temperature'] < 75) & (weather_data['Humidity'] < 40) & (weather_data['Wind'] < 6) ]
weather_data_ideal

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind,Cloudiness,Datetime
21,saint george,US,37.10,-113.58,68.23,21,4.70,1,1587068949
129,opuwo,NaN,-18.06,13.84,73.42,22,3.20,22,1587069065
222,avera,US,33.19,-82.53,70.11,28,5.82,1,1587069012
235,tabas,IR,33.60,56.92,66.20,39,2.24,8,1587068946
279,laguna,US,38.42,-121.42,74.44,38,5.82,1,1587069070
283,panzhihua,CN,26.59,101.71,64.42,32,1.12,0,1587069169
347,orangeburg,US,33.49,-80.86,69.12,22,3.36,1,1587069180
411,huntsville,US,34.73,-86.59,65.66,39,2.04,1,1587069195
423,simao,CN,22.79,100.97,63.86,37,2.15,33,1587069197


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
hotel_df = pd.DataFrame()
hotel_df["Hotel Name"] = ""
hotel_df["lat"] = ""
hotel_df["lng"] = ""
hotel_df["City"] = ""
hotel_df["Country"] = ""

weather_data_ideal["Hotel Name"] = np.nan

weather_data_ideal

C:\Users\hboll\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Latitude,Longitude,Temperature,Humidity,Wind,Cloudiness,Datetime,Hotel Name
21,saint george,US,37.10,-113.58,68.23,21,4.70,1,1587068949,NaN
129,opuwo,NaN,-18.06,13.84,73.42,22,3.20,22,1587069065,NaN
222,avera,US,33.19,-82.53,70.11,28,5.82,1,1587069012,NaN
235,tabas,IR,33.60,56.92,66.20,39,2.24,8,1587068946,NaN
279,laguna,US,38.42,-121.42,74.44,38,5.82,1,1587069070,NaN
283,panzhihua,CN,26.59,101.71,64.42,32,1.12,0,1587069169,NaN
347,orangeburg,US,33.49,-80.86,69.12,22,3.36,1,1587069180,NaN
411,huntsville,US,34.73,-86.59,65.66,39,2.04,1,1587069195,NaN
423,simao,CN,22.79,100.97,63.86,37,2.15,33,1587069197,NaN


In [105]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng
for index, row in weather_data_ideal.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][1]["name"]
        hotel_df.loc[index, 'lat'] = name_address["results"][1]['geometry']['location']['lat']
        hotel_df.loc[index, 'lng'] = name_address["results"][1]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = weather_data_ideal.loc[index, "City"]
        hotel_df.loc[index, 'Country'] = weather_data_ideal.loc[index, "Country"]
        weather_data_ideal.loc[index, "Hotel Name"] = name_address["results"][1]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

C:\Users\hboll\Anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Hotel Name,lat,lng,City,Country
21,Best Western Plus Abbey Inn,37.089,-113.584,saint george,US
129,PEP Opuwo,-18.0601,13.8412,opuwo,NaN
222,Georgia Life and Health Agency Inc,33.194,-82.5271,avera,US
235,کافی نت طبس آنلاین,33.596,56.9284,tabas,IR
279,Holiday Inn Express & Suites Elk Grove Central...,38.4241,-121.399,laguna,US
283,Nongnonggou,26.6053,101.686,panzhihua,CN
347,New York's Famous Hotdogs,33.4918,-80.8556,orangeburg,US
411,Embassy Suites by Hilton Huntsville Hotel & Spa,34.7249,-86.5901,huntsville,US
423,Pu'er Bureau of Land and Resources,22.7668,100.967,simao,CN


In [111]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]


In [112]:
# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,
    
    info_box_content= hotel_info
    #[f"{hotel} in {city}" for hotel in hotel_df["Hotel Name"]]
    #info_box_content=[f"Hotel: {hotel}" for hotel in hotel_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

#fig

# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))